In [1]:
import os
import sys
import pandas
import numpy

import findspark
findspark.init("/usr/local/spark/spark")

import pyspark

from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vector, Vectors

In [64]:
import matplotlib.pyplot as plt

In [3]:
file_name = "/Users/simondi/Desktop/test_ba/kmeans_transform-cells_sample_10_normalized_cut_100_K005"

In [4]:
conf = pyspark.SparkConf().setMaster("local[*]").set("spark.driver.memory", "4G").set("spark.executor.memory", "4G")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [5]:
data = spark.read.parquet(file_name)

In [6]:
data = data.limit(30)

In [7]:
data.take(5)

[Row(study='infectx_published', pathogen='listeria', library='a', design='u', replicate='1', plate='kb2-02-1x', well='a01', gene='chka', sirna='s3008', well_type='sirna', image_idx='4', object_idx='144', prediction=0, features=DenseVector([-0.8044, 0.0121, 1.1159, 1.4749, -0.9369, -0.7485, -1.0209, -0.703, 0.0, 0.4476, 1.2809, 1.3916, 0.1489, 0.6694, -1.2335, -0.0825, 0.0106, -0.5078, 1.2455, 0.1357])),
 Row(study='infectx_published', pathogen='listeria', library='a', design='u', replicate='1', plate='kb2-02-1x', well='a01', gene='chka', sirna='s3008', well_type='sirna', image_idx='5', object_idx='168', prediction=0, features=DenseVector([-0.9474, -0.0399, 0.1262, 0.6198, -1.0081, -0.7959, -0.9485, -0.6324, 0.0, 0.4263, -0.2104, -0.3939, 0.3941, -0.3786, 1.0053, -0.6748, 0.268, -1.0621, -0.5899, 0.429])),
 Row(study='infectx_published', pathogen='listeria', library='a', design='u', replicate='1', plate='kb2-02-1x', well='a01', gene='chka', sirna='s3008', well_type='sirna', image_idx='2

In [8]:
pca = PCA(k=2, inputCol="features", outputCol="pcs")

In [9]:
model = pca.fit(data)

In [10]:
data = model.transform(data)

In [11]:
data.take(1)

[Row(study='infectx_published', pathogen='listeria', library='a', design='u', replicate='1', plate='kb2-02-1x', well='a01', gene='chka', sirna='s3008', well_type='sirna', image_idx='4', object_idx='144', prediction=0, features=DenseVector([-0.8044, 0.0121, 1.1159, 1.4749, -0.9369, -0.7485, -1.0209, -0.703, 0.0, 0.4476, 1.2809, 1.3916, 0.1489, 0.6694, -1.2335, -0.0825, 0.0106, -0.5078, 1.2455, 0.1357]), pcs=DenseVector([1.7649, 1.6109]))]

In [16]:
counts = data.groupBy(["pathogen", "gene", "sirna"]).count()

+--------+----+-----+-----+
|pathogen|gene|sirna|count|
+--------+----+-----+-----+
|listeria|chka|s3008|   10|
|listeria|grk5|s6088|   10|
|listeria|chuk|s3077|   10|
+--------+----+-----+-----+



In [19]:
from pyspark.sql.window import Window

In [41]:
window = Window.partitionBy(["pathogen", "gene", "sirna"]).rowsBetween(0, 10)

In [45]:
from pyspark.sql.functions import row_number

In [53]:
data = data.withColumn("row_num", row_number().over(Window.partitionBy(["pathogen", "gene", "sirna"]).orderBy("pathogen")))

In [62]:
data = data.filter("row_num <= 10")

In [68]:
data.select(["pathogen", "gene", "sirna", ""])toPandas()

,study,pathogen,library,design,replicate,plate,well,gene,sirna,well_type,image_idx,object_idx,prediction,features,pcs,row_num
0,infectx_published,listeria,a,u,1,kb2-02-1x,a01,chka,s3008,sirna,4,144,0,"[-0.804423551995, 0.0121072635223, 1.115902332...","[1.76487012551, 1.61088575943]",1
1,infectx_published,listeria,a,u,1,kb2-02-1x,a01,chka,s3008,sirna,5,168,0,"[-0.947417075646, -0.0398719760855, 0.12623637...","[1.63468301131, 0.594261298524]",2
2,infectx_published,listeria,a,u,1,kb2-02-1x,a01,chka,s3008,sirna,2,210,2,"[-0.0838640082332, 1.12391127905, 1.7611178813...","[1.75772043796, -1.00642121456]",3
3,infectx_published,listeria,a,u,1,kb2-02-1x,a01,chka,s3008,sirna,6,207,4,"[-0.313931800476, 0.706510549442, -0.091321431...","[2.02350481383, 0.100451937143]",4
4,infectx_published,listeria,a,u,1,kb2-02-1x,a01,chka,s3008,sirna,2,243,2,"[-0.329908730493, -1.99159834871, 1.6817214535...","[2.25783110217, -2.77525388643]",5
5,infectx_published,listeria,a,u,1,kb2-02-1x,a01,chka,s3008,sirna,9,209,4,"[0.202522462319, -1.00675895612, -0.2317739851...","[1.53234179511, -1.49438655759]",6
6,infectx_published,listeria,a,u,1,kb2-02-1x,a01,chka,s3008,sirna,5,121,1,"[2.39216072113, -2.40321863194, -0.78612439023...","[-3.37821265856, -4.81525978572]",7
7,infectx_published,listeria,a,u,1,kb2-02-1x,a01,chka,s3008,sirna,2,212,4,"[-1.15951082162, -0.162684380858, 0.7271070824...","[2.81887922754, 0.382042590338]",8
8,infectx_published,listeria,a,u,1,kb2-02-1x,a01,chka,s3008,sirna,9,21,3,"[0.00440853011006, 1.28791352368, -1.246087393...","[-2.03657176404, 2.7128186201]",9
9,infectx_published,listeria,a,u,1,kb2-02-1x,a01,chka,s3008,sirna,3,178,4,"[-1.25177759247, 0.420635823377, -1.0334738603...","[2.16869401353, 1.386068262]",10


In [66]:
 font = {'weight': 'normal',
            'family': 'sans-serif',
            'size': 14}
plt.rc('font', **font)
plt.figure()
ax = plt.subplot(111)
plt.tick_params(axis="both", which="both", bottom="off", top="off",
                labelbottom="on", left="off", right="off", labelleft="on")
ax.spines["top"].set_visible(False)
ax.spines["bottom"].set_visible(True)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(True)  

In [ ]:
data

In [ ]:
  ax.plot(ks, score, "black")
    ax.plot(ks, score, "or")
    plt.xlabel('K', fontsize=15)
    plt.ylabel(axis_label, fontsize=15)
    plt.title('')
    ax.grid(True)
    logger.info("Saving plot to: {}".format(plotfile))
    plt.savefig(plotfile, bbox_inches="tight")